In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.integrate
import seaborn as sns
import holoviews as hv
import bokeh.io
import panel as pn

bokeh.io.output_notebook()
pn.extension('mathjax')
hv.extension('bokeh')

sns.set(font_scale=1.5, rc={"lines.linewidth": 2})

# Growth rate 
def r_temp(T, T_opt, r_max, sigma):
    return r_max * np.exp(-((T - T_opt) ** 2) / (2 * sigma ** 2))

# Gillespie simulation
def gillespie_temperature_growth(T, N0, K, r_max, T_opt, sigma, gamma, total_time):
    N = [N0]
    t = [0]
    
    while t[-1] < total_time:
        r = r_temp(T, T_opt, r_max, sigma)
        a_birth = r * N[-1] * (1 - N[-1] / K)
        a_death = gamma * N[-1]
        total_prop = a_birth + a_death
        
        if total_prop <= 0:
            break
        
        time_elapsed = np.random.exponential(1 / total_prop)
        t.append(t[-1] + time_elapsed)
        event = np.random.choice(["Birth", "Death"], p=[a_birth / total_prop, a_death / total_prop])
        if event == "Birth":
            N.append(N[-1] + 1)
        elif event == "Death" and N[-1] > 0:
            N.append(N[-1] - 1)
    
    return np.array(t), np.array(N)

def simulate_growth(T=37, sigma=5):
    # Parameters
    N0 = 10          # Initial population
    K = 1000         # Carrying capacity
    r_max = 0.1      # Max growth rate at optimal temperature
    T_opt = 37       # Optimal temperature (°C)
    gamma = 0.01     # Death rate
    total_time = 200 # Total simulation time
    
    # Gillespie simulation
    t, N = gillespie_temperature_growth(T, N0, K, r_max, T_opt, sigma, gamma, total_time)
    
    # Create Holoviews Curve
    curve = hv.Curve((t, N), 'Time (minutes)', 'Population Size').opts(
        width=800, height=400, title=f"Growth Simulation (T={T}°C, Sigma={sigma})",
        line_width=2, color='blue')
    
    return curve

slider_temp = pn.widgets.FloatSlider(name='Temperature (°C)', start=20, end=50, value=37, step=1)
slider_sigma = pn.widgets.FloatSlider(name='Sigma', start=1, end=10, value=5, step=0.5)

# Bind sliders to the simulation function
interactive_plot = pn.bind(simulate_growth, T=slider_temp, sigma=slider_sigma)

plot = pn.Column(slider_temp, slider_sigma, interactive_plot)
plot


Loading BokehJS ...

BokehModel(combine_events=True, render_bundle={'docs_json': {'7075c628-8d3a-4562-97dd-db78a18fd1b9': {'version…